# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [105]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [106]:
# checking your current working directory
print("Current working directory: " + os.getcwd())
print(' ')

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

    #print(file_path_list)
    print('File path list sample: ')
    print(file_path_list[0:5])

Current working directory: /workspace/home
 
File path list sample: 
['/workspace/home/event_data/2018-11-09-events.csv', '/workspace/home/event_data/2018-11-03-events.csv', '/workspace/home/event_data/2018-11-12-events.csv', '/workspace/home/event_data/2018-11-15-events.csv', '/workspace/home/event_data/2018-11-22-events.csv']


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [107]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print('Total number of rows: '+ str(len(full_data_rows_list)))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(' ')
print('Preview of first row: ')
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

        

Total number of rows: 8056
 
Preview of first row: 
['Muse', 'Logged In', 'Harper', 'M', '1', 'Barrett', '209.50159', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Supermassive Black Hole (Twilight Soundtrack Version)', '200', '1.54172E+12', '42']


In [108]:
# check the number of rows in your csv file
print("Number of rows in CSV file: ")
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

Number of rows in CSV file: 
6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [109]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster()

# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)
    
print("Completed Successfully")    

Completed Successfully


#### Create Keyspace

In [110]:
try:
    session.execute("""create keyspace if not exists music_data with replication = {'class':'SimpleStrategy', 'replication_factor':1} """)
except Exception as e:
    print (e)
    
print("Completed Successfully")    

Completed Successfully


#### Set Keyspace

In [111]:
try:
    session.set_keyspace('music_data')
except Exception as e:
    print (e)
    
print("Completed Successfully")    

Completed Successfully


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

# Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [112]:
# drop table if exists 
query1 = "drop table if exists session_artist_song"
try:
    session.execute(query1)
except Exception as e:
    print(e)

# create table for query 1
query2 = "create table if not exists session_artist_song "
query2 = query2 + "(sessionId int, itemInSession int, artist text, song text, length float, primary key (sessionId, itemInSession))"

try:
    session.execute(query2)
except Exception as e:
    print(e)        
    
print("Completed Successfully")    

Completed Successfully


In [113]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        ## Assign the INSERT statements into the `query` variable
        query = "insert into session_artist_song (sessionId, itemInSession, artist, song, length) "
        query = query + "values (%s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
print("Completed Successfully")    

Completed Successfully


SELECT statement to verify that the data have been inserted into each table

In [114]:
query = "select artist, song, length from session_artist_song where sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [ ]:
# drop table if exists 
query1 = "drop table if exists artist_song_user"
try:
    session.execute(query1)
except Exception as e:
    print(e)

# create table 
query2 = "create table if not exists artist_song_user "
query2 = query2 + "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, primary key ((userid, sessionid), iteminsession)) WITH CLUSTERING ORDER BY (iteminsession ASC);"

try:
    session.execute(query2)
except Exception as e:
    print(e)              

# Retrieve data from new csv and insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        ## Assign the INSERT statements into the `query` variable
        query = "insert into artist_song_user (userid, sessionid, iteminsession, artist, song, firstname, lastname) "
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))    

SELECT statement to verify the data was entered into the table

In [ ]:
query = "select artist, song, firstname, lastname from artist_song_user where userid = 10 and sessionid = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

# Query 3:  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [101]:
# drop table if exists 
query1 = "drop table if exists song_user"
try:
    session.execute(query1)
except Exception as e:
    print(e)

# create table  with song as partition key so that all users are stored on same node and user id 
# as clustering column to ensure all unique users are included in order
query2 = "create table if not exists song_user "
query2 = query2 + "(song text,  userid int, firstname text, lastname text,  PRIMARY KEY (song, userid))  WITH CLUSTERING ORDER BY (userid DESC)"

try:
    session.execute(query2)
except Exception as e:
    print(e)              

# Retrieve data from new csv and insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into song_user (userid, song, firstname, lastname) "
        query = query + "values (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))

print("Completed Successfully")           

Completed Successfully


SELECT statement to verify the data was entered into the table

In [102]:
## Add in the 
query = "select userid, firstname, lastname from song_user where song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print( row.userid, row.firstname, row.lastname)

95 Sara Johnson
80 Tegan Levine
29 Jacqueline Lynch


### Drop the tables before closing out the sessions

In [103]:
query1 = "drop table if exists session_artist_song"
try:
    session.execute(query1)
except Exception as e:
    print(e)

query2 = "drop table if exists artist_song_user"
try:
    session.execute(query2)
except Exception as e:
    print(e)

query3 = "drop table if exists song_user"
try:
    session.execute(query3)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [104]:
session.shutdown()
cluster.shutdown()